## Regression

In [ ]:
import os 
import sklearn
import numpy as np
import pandas as pd

from os import path as filepath
#from pyspark import SparkConf, SparkContext
#from pyspark.sql import SQLContext

In [ ]:
# Fixtures 
HDFS = "hdfs://{}".format(os.environ["HDFS"])
USER = filepath.join(HDFS, "user", "ec2-user")
RED_WINE = filepath.join(USER, "winequality/winequality-red.csv")
WHITE_WINE = filepath.join(USER, "winequality/winequality-white.csv")

In [22]:
def parse(line, sep=';'):
    """
    Parses a line based on separator
    """
    fields = line.strip().split(sep)
    try:
        return map(float, fields)
    except:
        return None #Header line will result in None

red_wine = sc.textFile(RED_WINE).map(parse)

#Filter out None resulting from first line
red_wine = red_wine.filter(lambda r: r is not None)

In [23]:
red_wine.take(10)

[[7.4, 0.7, 0.0, 1.9, 0.076, 11.0, 34.0, 0.9978, 3.51, 0.56, 9.4, 5.0],
 [7.8, 0.88, 0.0, 2.6, 0.098, 25.0, 67.0, 0.9968, 3.2, 0.68, 9.8, 5.0],
 [7.8, 0.76, 0.04, 2.3, 0.092, 15.0, 54.0, 0.997, 3.26, 0.65, 9.8, 5.0],
 [11.2, 0.28, 0.56, 1.9, 0.075, 17.0, 60.0, 0.998, 3.16, 0.58, 9.8, 6.0],
 [7.4, 0.7, 0.0, 1.9, 0.076, 11.0, 34.0, 0.9978, 3.51, 0.56, 9.4, 5.0],
 [7.4, 0.66, 0.0, 1.8, 0.075, 13.0, 40.0, 0.9978, 3.51, 0.56, 9.4, 5.0],
 [7.9, 0.6, 0.06, 1.6, 0.069, 15.0, 59.0, 0.9964, 3.3, 0.46, 9.4, 5.0],
 [7.3, 0.65, 0.0, 1.2, 0.065, 15.0, 21.0, 0.9946, 3.39, 0.47, 10.0, 7.0],
 [7.8, 0.58, 0.02, 2.0, 0.073, 9.0, 18.0, 0.9968, 3.36, 0.57, 9.5, 7.0],
 [7.5, 0.5, 0.36, 6.1, 0.071, 17.0, 102.0, 0.9978, 3.35, 0.8, 10.5, 5.0]]

In [24]:
X = red_wine.map(lambda r: r[:-1])
Y = red_wine.map(lambda r: r[-1])